In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from keras.models import Model, Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Input, Dense, Activation

Using TensorFlow backend.


In [23]:
#NOTE: none of these fxns are meant for single point input, input should always be a pd.DataFrame with more
#than one row, not an unreasonable request since sklearn makes you reshape 1D arrays.


# Data cleaning section: 

def split_and_scale(df, n):
    """Splits training dataframe into predictors and properties to be predicted and returns them in 2 new dfs.
       This function assumes all of the predictors are grouped together on the right side of the df.
       df_train: training df
       n: number of properties to be predicted(number of outputs)"""
    properties, predictors = split(df, n)
    # COMMENT OUT THIS LINE IF YOU DONT WANT TO HAVE POLYNOMIAL TERMS IN YOUR TRAINING DATA
    # But note that accuracy is much better with this, but the model will have higher variance
    predictors_polynomial = polynomialize(predictors)
    predictors_scaled_polynomial, predictors_scaler_polynomial = scaling(predictors_polynomial)
    return properties, predictors_scaled_polynomial, predictors_scaler_polynomial 


def polynomialize(series):
    """Adds polynomial features to degree 3, including interaction features. 
    series: an input ndarray of floats to be polynomialized.
    This function returns a ndarray of all of the features specified above."""
    # Creating polynomial object
    poly = PolynomialFeatures(degree = 2)
    # Adding polynomial terms
    predictors_polynomial = poly.fit_transform(series)
    return predictors_polynomial

# Still in development, in case we want to add more terms that aren't polynomial
# def add_nonlinear_terms(df, n):
#     properties = df[df.columns[-n:]]
#     predictors = df[df.columns[:-n]]
#     i = np.arange(len(predictors.columns) * 4)
#     x = 0
#     for column in predictors.values:
#         predictors.assign(i[x]=column**2)
#         predictors.assign(column**3)
#         predictors.assign(np.exp(column))
#         predictors.assign(np.sign(column))
#     return properties, predictors


def split(df, n):
    """Takes an input pd.DataFrame and returns 2 ndarrays of the properties 
    and predictors."""
    properties = df[df.columns[-n:]].values
    predictors = df[df.columns[:-n]].values
    return properties, predictors


def scaling(df_train):
    """This function takes a pd.DataFrame, creates a sklearn.StandardScaler, scales the DataFrame,
       and returns the scaled data in a pd.DataFrame as well as the sklearn.StandardScaler object
       for transforming data back to unscaled form post machine learning.
       df_train: pd.DataFrame(for our purposes should be of shape 20 columns by an arbitrary number of rows)"""
    #Creating scaler object
    scaler = preprocessing.MinMaxScaler()
    #Scaling df_train
    scaled_data = pd.DataFrame(scaler.fit_transform(df_train))
    
    return scaled_data, scaler

# Training/predicting


def train_model(df_train, df_validation, model, n):
    """This function takes a training DataFrame, validation DataFrame and a preconfigured model
       and trains said model on the training data followed by measuring error on validation data and 
       returning both the trained model and accuracy metric. This function assumes whatever parameter(s)
       being predicted is in the last column(s) of df_train.
       n: number of outputs
       because this function returns the trained model, more metrics can be performed later that are specific
       to whatever package it is in/the type of model it is
       Parameters"""
    #generating scaled data and their respective scaler objects
    t_properties, t_predictors_scaled, t_predictors_scaler = split_and_scale(df_train, n)
    v_properties, v_predictors_scaled, v_predictors_scaler = split_and_scale(df_validation, n)
    #supervised learning of predictors and properties to fit model, note: keras does not take pd.DataFrames for
    #training, using .values fixes this
    model.fit(t_predictors_scaled, t_properties)
    #predicting output of validation set
    predictions = pd.DataFrame(model.predict(v_predictors_scaled))
    #calculating RMSE from sklearn package
    val_error = np.sqrt(metrics.mean_squared_error(predictions, v_properties))
    return model, val_error, t_predictors_scaler


def model_prediction(test_data, fitted_model, scaler, n):
    """Takes a fitted model and predicts the output of test data, returns the predicted data and accuracy.
       THIS FUNCTION IS ONLY TO BE USED FOR FUTURE PREDICTIONS OR TESTING(WHICH SHOULD ONLY BE DONE ONCE).
       Do not use this while training a model, that's what the validation data will be used for. We do not 
       want to introduce bias into our model by fitting to the test data
       n = number of predictors"""
    #splitting predictors and properties
    properties, predictors = split(test_data, n)
    predictors = polynomialize(predictors)
    predictors_scaled = scaler.transform(predictors)
    #predicting based on scaled input predictors
    prediction = fitted_model.predict(predictors_scaled)
    #calculating MSE
    accuracy_metric = np.sqrt(metrics.mean_squared_error(properties, prediction))

    return prediction, accuracy_metric

# Below functions initialize all the different types of models we are looking at:


def neural_network():
    """Creates a neural network object to be passed into train_model function, can change properties of net
       here."""
    def model():
        model = Sequential()
        model.add(Dense(84, input_dim=84, kernel_initializer='normal', activation='relu'))
        model.add(Dense(84, kernel_initializer='normal', activation='relu'))
        model.add(Dense(1, kernel_initializer = 'normal'))#kernel_initializer = initial values of outputs i think
        model.compile(optimizer='adam', loss='mse')
        return model
    network = KerasRegressor(build_fn=model, epochs=100, batch_size=25000, verbose=0)
#     network.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, 
    return network


def linear_regression():
    """creates a linear regression object"""
    regr = LinearRegression()
    return regr



In [12]:
# to go in cleaning section
# import clean

def test_split():
    data = {'column1': [2, 2, 3], 'column2': [1, 3, 5]}
    df = pd.DataFrame(data)
    one, two = clean.split(df, 1)
    assert one[0] == 1
    assert two[0] == 2
    return

def test_scaling():
    data = {'column1': [2.0, 2.0, 3.0], 'column2': [1.0, 3.0, 5.0]}
    df = pd.DataFrame(data)
    df, scaler = clean.scaling(df)
    assert df.loc[0].iloc[0] == 0
    assert df.loc[2].iloc[0] == 1
    return



In [8]:
# TO REDUCE OVERFITTING: reduce degree of polynomial terms

In [9]:
#model might just not have the right things
#to reorder columns:
#cols = df.columns.tolist()
#df = df[cols] 

In [6]:
import numpy as np
import pandas as pd
from matminer.data_retrieval.retrieve_MP import MPDataRetrieval
mpdr = MPDataRetrieval(api_key='x5He3oeSg1eCaIU4')

In [7]:
# This statements obtains and stores the relevant data from MPD
# NOTE: Si was used as the criteria only for testing purposes. It will be changed later on
MPD_data = mpdr.get_dataframe(criteria='Si', properties=['xrd', 'band_gap', 'efermi'])

def extract_data(MPD_data_row):
    """
    Extracts the relevant XRD data from the dictionary obtained from MPD
    
    Parameters:
    ----------
    MPD_data_row : Pandas dataframe
         A row of data for a single material from the full MPD dataframe 
    
    Returns:
    ----------
    clean_df: Pandas dataframe
        The top 10 XRD peaks and their corresponding two theta values for the material
    """
    
    # Extracting out the amplitude and two theta values from the dictionary contained inside the received data
    # then turning it into a pandas dataframe.
    dirty_df = pd.DataFrame(MPD_data_row['xrd']['Cu']['pattern'], columns=MPD_data_row['xrd']['Cu']['meta']) # Converts data into dataframe
    dirty_df.drop(['hkl','d_spacing'], axis=1, inplace=True) # Disposes of the hkl and d-spacing data

    # Sorting the peaks into the top 10 with the highest peaks
    dirty_df.sort_values('amplitude', ascending=False, inplace=True) # Sorts peaks from highest to smallest
    dirty_df.reset_index(drop=True, inplace=True) # Reseting index
    clean_df = dirty_df[:10] # Dropping all peaks below the top ten 

    return clean_df

# Function to reformat the data after cleaning
# Takes the dataframe and turns it into a dictionary wwhere all data points have a unique key
def reformat_data(MPD_data_row):
    """
    Reformats the cleaned data obtained from the extract_data function into a dictionary
    
    Parameters:
    ----------
    MPD_data_row : Pandas dataframe
         A row of data for a single material from the full MPD dataframe 
    
    Returns:
    ----------
    clean_df: Pandas dataframe
        The top 10 XRD peaks and their corresponding two theta values for the material
    """
    
    # Cleaning data and creating empty dictionary
    clean_df = extract_data(MPD_data_row)
    mat_dict = {}

    # Loop to assign each data point to a key and stores it within the dictionary
    for i in range(0,20):
        if i < 10:
            amp_key = ('amplitude_' + str(i))
            mat_dict[amp_key] = clean_df['amplitude'][i]

        else:
            theta_key = ('two_theta_' + str(i-10))
            mat_dict[theta_key] = clean_df['two_theta'][i-10]

    return mat_dict

# Function 
def produce_data(MPD_data):
    """
    Produces the XRD and DOS data for all the materials passed to the function 
    
    Parameters:
    ----------
    MPD_data : Pandas dataframe
      The dataframe filled with data obtained from MPD 
    
    Returns:
    ----------
    full_df: Pandas dataframe
        The peaks, two theta values, band gap, and fermi energy for all the materials passed to the function
    """
    
    # Creating prelimanry containers for XRD and DOS data
    xrd_data = {}
    dos_data = MPD_data.drop(['xrd'], axis=1)
    
    # Loop to run through each row of the dataframe
    for i in range(len(MPD_data)):
        
        # Conditional to skip over materials with less than 10 XRD peaks
        # or no fermi energies
        if len(MPD_data.iloc[i]['xrd']['Cu']['pattern']) >= 10 and np.isnan(MPD_data.iloc[i]['efermi']) == False:
            
            # Obtaining and storing the XRD data for a material into a dictionary
            ID = MPD_data.index[i]
            mat_dict = reformat_data(MPD_data.iloc[i])
            xrd_data[ID] = mat_dict
            
        else:
            
            # Replaces rows that failed the conditional with NaN
            # This is for easy removal od the rows
            dos_data.iloc[i] = float('nan')
    
    # Creating the final dataframe from the obtained XRD and DOS dataframes
    dos_df = dos_data.dropna()
    xrd_df = pd.DataFrame.from_dict(xrd_data, orient='index')
    full_df = pd.concat([xrd_df, dos_df], axis=1, sort=False)
    
    return full_df

produce_data(MPD_data)

,amplitude_0,amplitude_1,amplitude_2,amplitude_3,amplitude_4,amplitude_5,amplitude_6,amplitude_7,amplitude_8,amplitude_9,...,two_theta_2,two_theta_3,two_theta_4,two_theta_5,two_theta_6,two_theta_7,two_theta_8,two_theta_9,band_gap,efermi
mp-1001113,100.0,78.288443,45.261343,40.440821,30.707488,26.515207,22.092113,21.737670,16.821364,16.690467,...,38.160738,171.874808,45.443514,176.442253,60.797841,148.648332,87.640251,80.839115,0.0000,9.167451
mp-1056579,100.0,91.828984,79.970463,45.423897,30.872127,26.312484,23.760237,15.234896,14.045885,13.052846,...,37.945623,73.006051,140.704562,54.746898,123.597176,85.886608,135.942882,93.269152,0.0000,10.142574
mp-1072544,100.0,74.250236,32.864700,32.201109,18.813035,17.053640,15.411324,14.668014,14.527636,13.937115,...,46.743236,32.580631,160.193335,50.741974,54.524230,42.462094,26.480052,98.859603,0.1444,3.791856
mp-109,100.0,71.345225,64.937703,44.646425,31.309877,23.430551,18.238651,16.929197,15.766726,15.187868,...,36.713745,38.289916,54.089719,38.027277,90.802389,155.580355,162.010286,169.996134,0.0000,9.233677
mp-149,100.0,66.810009,39.697299,23.446055,19.431330,17.901536,16.361733,13.659079,12.509826,11.039952,...,55.749540,87.355760,126.141124,113.020106,75.826656,94.191997,155.193701,135.154370,0.6119,5.634221
mp-16220,100.0,91.194330,82.558929,64.178610,58.432972,54.837508,39.433914,35.692211,30.283545,28.405679,...,52.695561,43.866856,17.015078,20.877953,170.458021,34.420518,165.871291,153.977258,0.5334,4.097343
mp-165,100.0,76.453355,75.130992,62.759221,57.677657,51.843314,44.605352,32.784208,32.075634,30.306187,...,47.209853,30.257230,28.034159,166.986591,55.806977,39.124870,90.702622,130.816041,0.4517,5.898865
mp-168,100.0,70.590906,66.583676,24.133148,18.174017,17.474408,14.443226,13.804826,11.952374,11.798358,...,170.276770,91.125960,26.789507,55.202796,151.438382,116.678065,78.754528,140.441238,0.0000,7.052171
mp-571520,100.0,43.334802,40.005817,32.918980,32.076427,30.139918,29.994833,25.861406,24.983342,21.405271,...,32.854949,51.589783,51.187368,170.098781,55.391229,52.089674,168.027021,51.288184,0.0000,7.133047
mp-676011,100.0,63.941827,37.274526,36.587043,34.737404,33.876036,30.127408,28.958163,28.511417,27.622809,...,27.177174,32.059515,45.915358,36.283100,38.803881,51.718617,20.404388,35.420940,0.0000,6.383710


In [28]:
# make sure inputs are correct, thats where shit gets fucked up
d_train = produce_data(MPD_data)
model, val_error, scaler = train_model(d_train.iloc[0:10], d_train.iloc[10:20],linear_regression(), 2)
predictions, accuracy = model_prediction(d_train.iloc[0:2], model, scaler, 2)

In [29]:
predictions

array([[-5.55111512e-17,  9.16745132e+00],
       [ 8.32667268e-17,  1.01425740e+01]])

In [30]:
accuracy

1.2570702125854891e-15